In [ ]:
# In this notebook I will filter the csv files; year and tarra from the ocr results
# Note: The filter could be optimized further for the tarra column, since there are now too many empty tarra columns (look into the data for that but will do later)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import re

In [61]:
df = pd.read_csv('bottle_ocr_results_13_44.csv')

# pattern_tarra = r"(1\d[.,\s]?\d{1,2})"
pattern_tarra = r"(1[0-7][.,\s]?\d{1,2}?)"
pattern_year = r"(20\d{2})"

# Filter out the patterns
extracted_tarra = df['ocr_text'].str.extract(pattern_tarra, expand=False)
extracted_year = df['ocr_text'].str.extract(pattern_year, expand=False)

# Create dataframe for tarra and year (for the counting later)
df_extracted_tarra = df[['bottle_id', 'ocr_text']].copy()
df_extracted_tarra['extracted_value'] = extracted_tarra
df_extracted_tarra = df_extracted_tarra.dropna(subset=['extracted_value'])

df_extracted_year = df[['bottle_id', 'ocr_text']].copy()
df_extracted_year['extracted_value'] = extracted_year
df_extracted_year = df_extracted_year.dropna(subset=['extracted_value'])

# View the results for verifying the "correctness"
print("Extracted Tarra values:")
print(df_extracted_tarra)
print("\nExtracted Year values:")
print(df_extracted_year)


Extracted Tarra values:
      bottle_id                         ocr_text extracted_value
2             0                     TARE 10.66KC            10.6
10            0       4 UN1965 1219 615 2039 " W             121
12            1            2215 4 119 P1SY MAGAZ            15 4
31            2                           15,4K6            15,4
83            5                10.5 3 Un196' KG-            10.5
...         ...                              ...             ...
2637        147                              147             147
2645        148                          o 615 0            15 0
2647        148  PROPAAN €15 8KG 2037 2 965 '10,            15 8
2658        148                       147 MMAGAZ             147
2717        151                       €15 10J, 4            15 1

[252 rows x 3 columns]

Extracted Year values:
      bottle_id                          ocr_text extracted_value
4             1                       2039 [ TARE            2039
5             1 

In [62]:
# Get the most frequent tarra value per bottle_id
tarra_counts = df_extracted_tarra.groupby('bottle_id')['extracted_value'].value_counts()
most_frequent_tarra = tarra_counts.groupby('bottle_id').nlargest(1).reset_index(level=0, drop=True).reset_index()
most_frequent_tarra = most_frequent_tarra.rename(columns={'extracted_value': 'tarra', 'count': 'tarra_count'})

# Get the most frequent year value per bottle_id
year_counts = df_extracted_year.groupby('bottle_id')['extracted_value'].value_counts()
most_frequent_year = year_counts.groupby('bottle_id').nlargest(1).reset_index(level=0, drop=True).reset_index()
most_frequent_year = most_frequent_year.rename(columns={'extracted_value': 'year', 'count': 'year_count'})

# Create final dataframe with all bottle_ids
all_bottle_ids = df['bottle_id'].unique()
final_df = pd.DataFrame({'bottle_id': all_bottle_ids})

# Merge with tarra and year data (left join to keep all bottle_ids)
final_df = final_df.merge(most_frequent_tarra[['bottle_id', 'tarra']], on='bottle_id', how='left')
final_df = final_df.merge(most_frequent_year[['bottle_id', 'year']], on='bottle_id', how='left')

In [ ]:
# Normalize tarra values with last digit as decimal and REPLACE the original tarra column
def normalize_tarra(tarra_value):
    if pd.isna(tarra_value):
        return tarra_value
    
    # Remove any whitespace and convert to string
    tarra_value = str(tarra_value).strip()
    
    # Remove any existing dots, commas, or spaces first
    clean_value = tarra_value.replace('.', '').replace(',', '').replace(' ', '')
    
    # Handle the case where we have 3 digits (like 109, 121)
    if len(clean_value) == 3:
        # Treat as XX.X format (last digit is decimal)
        integer_part = clean_value[:2]  # First two digits
        decimal_part = clean_value[2]   # Last digit
        return f"{integer_part}.{decimal_part}0"
    
    # Handle the case where we have 2 digits (like 16)
    elif len(clean_value) == 2:
        return f"{clean_value}.00"
    
    # Handle cases with existing separators
    elif '.' in tarra_value or ',' in tarra_value or ' ' in tarra_value:
        # Replace commas and spaces with dots
        normalized = tarra_value.replace(',', '.').replace(' ', '.')
        parts = normalized.split('.')
        
        if len(parts) >= 2:
            integer_part = parts[0].zfill(2)
            decimal_part = parts[1][:2].ljust(2, '0')  # Take up to 2 decimal digits
            return f"{integer_part}.{decimal_part}"
        else:
            return f"{parts[0].zfill(2)}.00"
    
    # Fallback for other cases
    else:
        return f"{clean_value.zfill(2)}.00"

#replaced the tarra column with normalized values instead of creating a new column
final_df['tarra'] = final_df['tarra'].apply(normalize_tarra)

print("\nFinal DataFrame:")
print(final_df)
final_df.to_csv('bottle_ocr_filtered_13_44.csv', index=False)


Final DataFrame:
     bottle_id  tarra  year
0            1  15.40  2039
1            0  10.60  2039
2            2  15.40  2055
3            3    NaN   NaN
4            4    NaN  2035
..         ...    ...   ...
150        150    NaN   NaN
151        151  15.10  2035
152        152    NaN   NaN
153        153    NaN   NaN
154        154    NaN   NaN

[155 rows x 3 columns]
